In [ ]:
# Importar as bibliotecas

import json
import os
import pandas
from pathlib import Path
import sys

In [ ]:
# Resolver os 'imports' do projeto 

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

from utils.classes.pandas_dataframe import PandasDataframe as pd_dataframe
import utils.functions.date_transform as date_transform

In [ ]:
# Traz as opções de configuração do JSON

json_file = os.path.abspath('../../options.json')

with open(json_file, 'r', encoding='utf-8') as j_file:
    json_data = json.load(j_file)

In [ ]:
# Declaração dos caminhos

dir_archive  = os.path.join(os.path.abspath('../../data'))
dir_cvm      = os.path.join(dir_archive, "cvm")
dir_cvm_csv  = os.path.join(dir_cvm, json_data['DIR']['CVM']['CSV_NAME'])
dir_cvm_data = os.path.join(dir_cvm, json_data['DIR']['CVM']['DATA_NAME'])

In [ ]:
# Lista os investimentos escolhidos

CHOOSED_INVESTMENTS = json_data["INVESTMENTS"]
CHOOSED_INVESTMENTS = list(CHOOSED_INVESTMENTS.keys())

In [ ]:
# Prepara variáveis para filtragem por data 

START_DATE = json_data['CONFIG']['START_DATE']
END_DATE   = json_data['CONFIG']['END_DATE']

In [ ]:
# Lista o histórico dos investimentos escolhidos

list_csv_archive = os.listdir(dir_cvm_csv)
list_csv_archive_filtered = []

if len(list_csv_archive) == 0:
    print('Não há arquivos na pasta "/app/data/"' + json_data['CONFIG']['DIR_CVM_CSV_NAME'])
    sys.exit()

for csv_archive_month in list_csv_archive: 
    list_investments_month_path = os.path.join(dir_cvm_csv, csv_archive_month)
    
    list_investments_month = pd_dataframe(list_investments_month_path, None)
    list_investments_month.csv_to_df()

    column_element = 'ID_SUBCLASSE'
    AXIS_ROW = 1

    list_investments_month.drop_column(column_element, AXIS_ROW)

    column_element = 'CNPJ_FUNDO_CLASSE'
    COLUMN_DATE    = 'DT_COMPTC'

    list_investments_month.query_date(START_DATE, END_DATE, COLUMN_DATE)
    list_investments_month.query_element_in(column_element, CHOOSED_INVESTMENTS)
    list_csv_archive_filtered.append(list_investments_month.df)

if len(list_csv_archive_filtered) == 0:
    print('Não há nenhum dado populado')
    sys.exit()

In [ ]:
# Tratamento dos dados concatenados

list_investments_filtered = pandas.concat(list_csv_archive_filtered, ignore_index=True)
list_investments_filtered = pd_dataframe(None, list_investments_filtered)

ORDER_ELEMENTS_LIST = ["CNPJ_FUNDO_CLASSE", "DT_COMPTC"]
list_investments_filtered.sort_elements_list(ORDER_ELEMENTS_LIST)
list_investments_filtered.reset_index()

In [ ]:
# Cria a coluna do nome dos fundos

def search_name(element): 
    # Eu não entendo essa sintaxe, mas de alguma forma ele pega o nome da coluna
    return json_data['INVESTMENTS'][element]['Fundo']

list_investments_filtered.df['NOME_FUNDOS'] = list_investments_filtered.df['CNPJ_FUNDO_CLASSE'].map(search_name)

In [ ]:
# Cria a coluna da valorização da(s) cota(s)

started_value_cota = 0
preview_cnpj = ''

def calculate_valuation(row):
    global started_value_cota
    global preview_cnpj

    # Tratar o caso de loop de CNPJ
    current_cnpj = row['CNPJ_FUNDO_CLASSE']
    if preview_cnpj != current_cnpj:
        started_value_cota = 0
    preview_cnpj = current_cnpj

    investment_date_start = json_data['INVESTMENTS'][row['CNPJ_FUNDO_CLASSE']]['HistoricoCompraDatas'][0]
    investment_date_row = str(row['DT_COMPTC'])[:10]

    date_object_start = date_transform.transform_date_string_on_number_list(investment_date_start)
    date_object_row   = date_transform.transform_date_string_on_number_list(investment_date_row)

    boolean_verification = date_transform.is_first_date_previous_than_second(date_object_start, date_object_row)

    if boolean_verification == False:
        # Isso pode dar erro, estou seguindo a regra que um usuário só pode comprar quando cotado o valor, pois se não, como afirmar qual foi a data?
        if investment_date_start == investment_date_row:
            started_value_cota = float(row['VL_QUOTA'])
        return 0.00
    
    valuation_percentual = ((float(row['VL_QUOTA']) / started_value_cota) - 1) * 100
    return round(valuation_percentual, 2)

list_investments_filtered.df['VALUATION_PERCENT'] = list_investments_filtered.df.apply(calculate_valuation, axis=1)

In [ ]:
# Ordena as colunas

ORDER_DF = ["CNPJ_FUNDO_CLASSE", "NOME_FUNDOS", "DT_COMPTC", "VL_QUOTA", "VALUATION_PERCENT", "VL_TOTAL","VL_PATRIM_LIQ", "CAPTC_DIA", "RESG_DIA","NR_COTST"]

list_investments_filtered.order_columns(ORDER_DF)

In [ ]:
# Devolve as tabelas pelos nomes dos investimentos

if not os.path.exists(dir_cvm_data):
    os.mkdir(dir_cvm_data)

list_investments_filtered.group_element('NOME_FUNDOS')
dataframe_by_nome_fundo = {}

for nome_fundo, data in list_investments_filtered.df:
    archive_name = nome_fundo.replace(' ', '_') + '.csv'
    cvm_path = os.path.join(dir_cvm_data, archive_name)

    dataframe_by_nome_fundo = pd_dataframe(None, data)
    dataframe_by_nome_fundo.df_to_csv(cvm_path)